This notebook implements a workflow to analyze point cloud data from two epochs captured before and after a rockfall event. The key steps include:

- Reading and loading two point cloud files (LAS/LAZ) representing the epochs.
- Creating Vapc objects (v1 and v2) for each epoch and computing the bitemporal Mahalanobis distance to identify significant changes.
- Filtering the points based on a significance mask and selecting only those points that show a significant change.
- Running the M3C2 algorithm on the significant points to compute distances and associated uncertainties.
- Integrating the distance and uncertainty values into the Vapc object's output and saving the results as a LAS file.

The notebook uses parameters such as voxel size, alpha, and minimum points to fine-tune the analysis. The results, including the computed distances and uncertainties, are stored in variables like distances, uncertainties, and points_with_change, and eventually saved to the designated output folder.

In [1]:
import py4dgeo
import numpy as np

before_rockfall_filepath = r"C:\Users\nc298\repos\py4dgeo\tests\data\240826_000005.laz"
after_rockfall_filepath = r"C:\Users\nc298\repos\py4dgeo\tests\data\240826_010006.laz"

before_rockfall_filepath = r"D:\daten\projects\aimon\georefernced_epochs\rockfall_low_res_2_epochs\ScanPos001 - SINGLESCANS - 240826_000005.las"
after_rockfall_filepath = r"D:\daten\projects\aimon\georefernced_epochs\rockfall_low_res_2_epochs\ScanPos001 - SINGLESCANS - 240826_010006.las"

outfolder = r"C:\Users\nc298\Downloads"
voxel_size = 6
alpha = .999
min_points = 30

epoch1, epoch2 = py4dgeo.read_from_las(before_rockfall_filepath, after_rockfall_filepath)


[2025-06-27 14:20:41][INFO] Reading point cloud from file 'D:\daten\projects\aimon\georefernced_epochs\rockfall_low_res_2_epochs\ScanPos001 - SINGLESCANS - 240826_000005.las'
[2025-06-27 14:20:42][INFO] Reading point cloud from file 'D:\daten\projects\aimon\georefernced_epochs\rockfall_low_res_2_epochs\ScanPos001 - SINGLESCANS - 240826_010006.las'


In [2]:
# Mute vapc function trace and timeit
py4dgeo.enable_trace(False)
py4dgeo.enable_timeit(False)

voxel_epoch1 = py4dgeo.Vapc(epoch1, voxel_size=voxel_size)
voxel_epoch2 = py4dgeo.Vapc(epoch2, voxel_size=voxel_size)

#Compute delta vapc
mahalanobis_result = voxel_epoch1.compute_bitemporal_mahalanobis(voxel_epoch2, alpha=alpha, min_points=min_points)

In [3]:
# Filter significant changes
# The significance is stored in the 'significance' field of the out dictionary
sig_filter = mahalanobis_result.out['significance'] == 1
# Apply the filter to the Vapc object
# This will keep only the significant changes in the Vapc object
mahalanobis_result.filter(sig_filter, overwrite=True)
# Select points with significant changes
# This will return a new Vapc object with only the points that have significant changes
voxel_epoch_1_with_significant_change = voxel_epoch1.select_by_mask(mahalanobis_result)

In [4]:
m3c2 = py4dgeo.M3C2(
    epochs=(epoch1, epoch2),
    corepoints=voxel_epoch_1_with_significant_change.epoch.cloud,
    cyl_radius=1.0,
    normal_radii=[1.],
    max_distance=10.0
)

distances, uncertainties = m3c2.run()

[2025-06-27 14:20:43][INFO] Building KDTree structure with leaf parameter 10
[2025-06-27 14:20:44][INFO] Building KDTree structure with leaf parameter 10


In [5]:
voxel_epoch_1_with_significant_change.out['distances'] = distances
d = {name: uncertainties[name] for name in uncertainties.dtype.names }
d_filtered = {k: v for k, v in d.items() if k != 'dtype'}
voxel_epoch_1_with_significant_change.out.update(d)
voxel_epoch_1_with_significant_change.out["significant_change"] = np.abs(distances) > uncertainties['lodetection']

#Save the Vapc object with m3c2 results
outfile = f"{outfolder}/mahalanobis_vapc_{voxel_size}_{alpha}_m3c2.laz"
voxel_epoch_1_with_significant_change.save_as_las(outfile)